In [1]:
import os, sys
%pwd

'/config/workspace/notebook'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path

In [4]:
from accident_severity.constants import *
from accident_severity.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self)-> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        

        return data_transformation_config

In [6]:
import os, sys 
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from accident_severity.logging import logger
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.preprocessor = None 
        self.transformed_df = None 


    def get_data_transformation(self):
        try:

            # Load the dataset
            df = pd.read_csv(self.config.data_path)

            # Renaming the columns
            col_map={
                'Time': 'time',
                'Day_of_week': 'day_of_week',
                'Age_band_of_driver': 'driver_age',
                'Sex_of_driver': 'driver_sex',
                'Educational_level': 'educational_level',
                'Vehicle_driver_relation': 'vehicle_driver_relation',
                'Driving_experience': 'driving_experience',
                'Type_of_vehicle': 'vehicle_type',
                'Owner_of_vehicle': 'vehicle_owner',
                'Service_year_of_vehicle': 'service_year',
                'Defect_of_vehicle': 'vehicle_defect',
                'Area_accident_occured': 'accident_area',
                'Lanes_or_Medians': 'lanes',
                'Road_allignment': 'road_allignment',
                'Types_of_Junction': 'junction_type',
                'Road_surface_type': 'surface_type',
                'Road_surface_conditions': 'road_surface_conditions',
                'Light_conditions': 'light_condition',
                'Weather_conditions': 'weather_condition',
                'Type_of_collision': 'collision_type',
                'Number_of_vehicles_involved': 'vehicles_involved',
                'Number_of_casualties': 'casualties',
                'Vehicle_movement': 'vehicle_movement',
                'Casualty_class': 'casualty_class',
                'Sex_of_casualty': 'casualty_sex' ,
                'Age_band_of_casualty': 'casualty_age',
                'Casualty_severity': 'casualty_severity',
                'Work_of_casuality': 'casualty_work',
                'Fitness_of_casuality': 'casualty_fitness',
                'Pedestrian_movement': 'pedestrian_movement',
                'Cause_of_accident': 'accident_cause',
                'Accident_severity': 'accident_severity'
            }
            df.rename(columns=col_map, inplace=True)

            # drop some columns
            df.drop(columns=["time"], axis=1, inplace=True) 

            # Define the target_variable
            X = df.drop(columns=["accident_severity"], axis=1)
            y = df["accident_severity"]

            # Map the target variable mannually
            y.replace({'Slight Injury': 0,
               'Serious Injury': 1,
               'Fatal injury': 2}, inplace=True)
            
            # Define numerical and categorical features

            numerical_features = X.select_dtypes(exclude="object").columns
            categorical_features = X.select_dtypes(include="object").columns

            # Define the pipeline
            num_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="median")),
                    ("robustscaler", RobustScaler())
                ]
            )

            cat_pipeline = Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="most_frequent")),
                    ("ordinalencoder", OrdinalEncoder(handle_unknown="error"))
                ]
            )

            logger.info(f"Numerical columns: {numerical_features}")
            logger.info(f"Categorical columns: {categorical_features}")

            # Define the Preprocessor

            preprocessor = ColumnTransformer(
                transformers=[
                    ("mum_pipeline", num_pipeline, numerical_features),
                    ("cat_pipeline", cat_pipeline, categorical_features)
                ]
            )

            self.preprocessor = preprocessor # Store the preprocessor for later usage

            # Transform the whole data using the preprocessor
            X_transformed = preprocessor.fit_transform(X)

            # Get the updated column names after ordinal encoding
            column_names = numerical_features.to_list() + categorical_features.to_list()

            # Combine X_transformed and y back into one Dataframe
            self.transformed_df = pd.DataFrame(X_transformed, columns=column_names)
            self.transformed_df["accident_severity"] = y 

            logger.info(f"Data preprocessing completed")

        except Exception as e:
            raise e

    def handle_data_imbalance(self):
        if self.transformed_df is None:
            raise ValueError("Data transformation is not available. Please call get_data_transformation.")

        # Split the data into train and test sets
        train, test = train_test_split(self.transformed_df)

        # Separate features and target in the train set
        X_train = train.drop(columns=["accident_severity"])
        y_train = train["accident_severity"]

        # Handle data imbalance using SMOTE
        smote = SMOTE(sampling_strategy='auto', random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        # Save the resampled train set in a CSV file
        train_resampled = pd.DataFrame(X_train_resampled, columns=X_train.columns)
        train_resampled["accident_severity"] = y_train_resampled
        train_resampled.to_csv(os.path.join(self.config.root_dir, "train_resampled.csv"), index=False)

        logger.info("Handling data imbalance using SMOTE completed")
        
    
    def train_test_split(self):
        if self.preprocessor is None:
            raise ValueError(f"Preprocessor is not available. Please call get_data_transformation")
        
        # Split the data into train and test sets
        train, test = train_test_split(self.transformed_df)


        # Save the encoded train and test sets in form of csv files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info(f"Spitted the data into train and test set")
        logger.info(f"Shape of train data: ", train.shape)
        logger.info(f"Shape of test data: ", test.shape)


In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transformation()
    data_transformation.handle_data_imbalance()
    data_transformation.train_test_split()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config/config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/data_transformation]
[{asctime}s: INFO: 3060276492: Numerical columns: Index(['vehicles_involved', 'casualties'], dtype='object')]
[{asctime}s: INFO: 3060276492: Categorical columns: Index(['day_of_week', 'driver_age', 'driver_sex', 'educational_level',
       'vehicle_driver_relation', 'driving_experience', 'vehicle_type',
       'vehicle_owner', 'service_year', 'vehicle_defect', 'accident_area',
       'lanes', 'road_allignment', 'junction_type', 'surface_type',
       'road_surface_conditions', 'light_condition', 'weather_condition',
       'collision_type', 'vehicle_movement', 'casualty_class', 'casualty_sex',
       'casualty_age', 'casualty_severi

--- Logging error ---
Traceback (most recent call last):
  File "/config/workspace/traffic/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/config/workspace/traffic/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/config/workspace/traffic/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/config/workspace/traffic/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/config/workspace/traffic/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/config/workspace/traffic/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/config/workspace/traffic/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_